In [2]:
import numpy as np 
import pandas as pd

In [3]:
df=pd.read_csv("Unclean_combined_dataset.csv")

In [4]:
df

,zpid,price,priceLabel,address,beds,baths,hdpData.homeInfo.livingArea,latLong.latitude,latLong.longitude,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,country,Unnamed: 15,Unnamed: 16
0,223325018,285000,NaN,Nhn Eastern Psge,4.0,3.0,560.00000,24.560984,-132.239400,99929.0,Wrangell,AK,3.400000,Acres,USA,NaN,NaN
1,227395822,"$650,000",$650K,"6525 N Douglas Hwy, Juneau, AK 99801",4.0,2.0,2399.00000,25.697077,-134.491990,99801.0,Juneau,AK,0.640000,acres,USA,NaN,NaN
2,230756778,620000,NaN,3479 Meander Way,4.0,3.0,1959.00000,26.025694,-134.594010,99801.0,Juneau,AK,0.255372,Acres,USA,NaN,NaN
3,230966374,"$295,000",$295K,"222 Huckleberry Ln, Dillingham, AK 99576",4.0,3.0,1300.00000,26.129187,-158.699620,99576.0,Dillingham,AK,1.610000,acres,USA,NaN,NaN
4,237711393,"$120,000",$120K,"Nhn Chilkat Lk, Haines, AK 99827",4.0,4.0,528.00000,26.132538,-135.885450,99827.0,Haines,AK,1.000000,acres,USA,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6305,544880,1606123.32,NaN,"7814 Kinghaven Dr, Little Elm, TX 75068, Richm...",7.0,6.0,3376.86599,69.747710,-102.902248,23226.0,Richmond,VA,9741.694358,sqft,USA,NaN,NaN
6306,45062898,"From $129,999",NaN,"3100 Jeanetta St, Houston, TX",3.0,2.0,NaN,64.898926,-95.525970,NaN,NaN,NaN,NaN,NaN,USA,NaN,NaN
6307,84405759,"From $699,900",NaN,"916 G St NW, Washington, DC",4.0,3.0,NaN,64.944780,-77.024910,NaN,NaN,NaN,NaN,NaN,USA,NaN,NaN
6308,109326667,"From $7,895,000",NaN,"217 W 57th St, New York, NY",4.0,4.0,NaN,65.130890,-73.980965,NaN,NaN,NaN,NaN,NaN,USA,NaN,NaN


In [5]:
df = df.drop_duplicates(subset=['latLong.latitude','latLong.longitude'])

In [6]:
df = df.rename(columns={ 'hdpData.homeInfo.livingArea': 'livingArea', 'hdpData.homeInfo.lotAreaValue': 'lotAreaValue', 'hdpData.homeInfo.lotAreaUnit': 'lotAreaUnit','hdpData.homeInfo.zipcode': 'zipcode','hdpData.homeInfo.city': 'city','hdpData.homeInfo.state': 'state' })

In [7]:
df['price'] = df['price'].astype(str).str.lstrip("$")

In [8]:
df['price'] = df['price'].astype(str).str.replace(",", "") 


In [9]:
df['price'] = df['price'].astype(str).str.replace(r'\D+', '', regex=True)
df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [10]:

df['lotAreaUnit'] = df['lotAreaUnit'].str.lower()


In [11]:
df['unit conversion'] = df['lotAreaUnit'].apply(lambda x:43560  if x=='acres' else 1 )


In [12]:
df = df.replace(0, np.nan)

In [13]:
total_nulls = df.isnull().sum()
print("Total null values:", total_nulls)
total_zeros = (df == 0).sum().sum() 
print("Total zero values:", total_zeros)


Total null values: zpid                    0
price                   8
priceLabel           5739
address                 0
beds                   10
baths                  15
livingArea             22
latLong.latitude        0
latLong.longitude       0
zipcode                 4
city                    4
state                   4
lotAreaValue          104
lotAreaUnit            92
country                 0
Unnamed: 15          6310
Unnamed: 16          6310
unit conversion         0
dtype: int64
Total zero values: 0


In [14]:
total_nulls = df['livingArea'].isnull().sum().sum() 
print("Total null values:", total_nulls)
total_zeros = (df['livingArea'] == 0).sum().sum() 
print("Total zero values:", total_zeros)

Total null values: 22
Total zero values: 0


In [15]:
df['lotAreaValue']=df['lotAreaValue']* df['unit conversion']

In [16]:
df['livingArea'] = df['livingArea'].fillna(df['lotAreaValue'] * 0.5)

In [17]:
df['lotAreaValue'] = df['lotAreaValue'].fillna(df['livingArea'] * 2)

In [18]:
total_nulls = df['livingArea'].isnull().sum().sum() 
print("Total null values:", total_nulls)

Total null values: 13


In [19]:
df['price Conversion'] = df['priceLabel'].apply(lambda x:1000 if "k" in str(x).lower() else 1000000 if "m" in str(x).lower() else 1)

In [20]:
df['priceLabel'] = df['priceLabel'].astype(str).str.replace(r'\D+', '', regex=True)
df['priceLabel'] = pd.to_numeric(df['priceLabel'], errors='coerce')

In [21]:
df['priceLabel']=df['priceLabel']* df['price Conversion']

In [22]:
total_nulls = df['price'].isnull().sum()
print("Total null values:", total_nulls)
total_zeros = (df['price'] == 0).sum().sum() 
print("Total zero values:", total_zeros)

Total null values: 8
Total zero values: 0


In [23]:
df['price'] = df['price'].fillna(df['priceLabel'])

In [24]:
df['price'] = np.floor(df['price'])

In [25]:
total_nulls = df['beds'].isnull().sum()
print("Total null values:", total_nulls)
total_zeros = (df['beds'] == 0).sum().sum() 
print("Total zero values:", total_zeros)

Total null values: 10
Total zero values: 0


In [26]:
beds_mode=df['beds'].mode()[0]
df['beds'] = df['beds'].fillna(beds_mode)

baths_mode=df['baths'].mode()[0]
df['baths'] = df['baths'].fillna(baths_mode)

In [27]:
total_nulls = df['beds'].isnull().sum()
print("Total null values:", total_nulls)
total_zeros = (df['beds'] == 0).sum().sum() 
print("Total zero values:", total_zeros)

Total null values: 0
Total zero values: 0


In [28]:
total_nulls = df['baths'].isnull().sum()
print("Total null values:", total_nulls)
total_zeros = (df['baths'] == 0).sum().sum() 
print("Total zero values:", total_zeros)

Total null values: 0
Total zero values: 0


In [29]:
total_nulls = df.isnull().sum()
print("Total null values:", total_nulls)
total_zeros = (df == 0).sum().sum() 
print("Total zero values:", total_zeros)


Total null values: zpid                    0
price                   8
priceLabel           5741
address                 0
beds                    0
baths                   0
livingArea             13
latLong.latitude        0
latLong.longitude       0
zipcode                 4
city                    4
state                   4
lotAreaValue           13
lotAreaUnit            92
country                 0
Unnamed: 15          6310
Unnamed: 16          6310
unit conversion         0
price Conversion        0
dtype: int64
Total zero values: 0


In [30]:
df= df[df['price'] >= 50000]

In [31]:
df= df[df['livingArea'] <= 20000]

In [32]:
df2 = df[['livingArea','beds', 'baths','address','zipcode','city','state','country','price']]

In [33]:
df2 = df2.rename(columns={ 'livingArea': 'livingArea in sq.ft','price':'price in $'})

In [34]:
df2.dropna()

,livingArea in sq.ft,beds,baths,address,zipcode,city,state,country,price in $
0,560.000000,4.0,3.0,Nhn Eastern Psge,99929.0,Wrangell,AK,USA,2.850000e+05
1,2399.000000,4.0,2.0,"6525 N Douglas Hwy, Juneau, AK 99801",99801.0,Juneau,AK,USA,6.500000e+05
2,1959.000000,4.0,3.0,3479 Meander Way,99801.0,Juneau,AK,USA,6.200000e+05
3,1300.000000,4.0,3.0,"222 Huckleberry Ln, Dillingham, AK 99576",99576.0,Dillingham,AK,USA,2.950000e+05
4,528.000000,4.0,4.0,"Nhn Chilkat Lk, Haines, AK 99827",99827.0,Haines,AK,USA,1.200000e+05
...,...,...,...,...,...,...,...,...,...
6301,2740.691602,6.0,5.0,"3535 Lois Dr, Henrico, VA 23229",23229.0,Henrico,VA,USA,6.233554e+06
6302,4179.479164,9.0,8.0,"5803 Clairmount St, Henrico, VA 23229",23229.0,Henrico,VA,USA,1.171319e+16
6303,3304.791536,7.0,6.0,"2293 Ontario Rd NW APT 3, Washington, DC 20009...",23229.0,Henrico,VA,USA,1.070231e+16
6304,1139.516210,1.0,1.0,"6525 Shepherd St NW, Washington, DC 20011, Ric...",23226.0,Richmond,VA,USA,3.016984e+15


In [35]:
df2['zipcode']=df2['zipcode'].astype('Int64')
df2['beds'] = pd.to_numeric(df['beds'], errors='coerce')
df2['baths'] = pd.to_numeric(df['baths'], errors='coerce')


In [36]:
df2['baths']=df2['baths'].astype('Float32')
df2['baths']=df2['baths'].astype('Int64')
df2['beds']=df2['beds'].astype('Int64')
df2['price in $']=df2['price in $'].astype('Int64')
df2

,livingArea in sq.ft,beds,baths,address,zipcode,city,state,country,price in $
0,560.000000,4,3,Nhn Eastern Psge,99929,Wrangell,AK,USA,285000
1,2399.000000,4,2,"6525 N Douglas Hwy, Juneau, AK 99801",99801,Juneau,AK,USA,650000
2,1959.000000,4,3,3479 Meander Way,99801,Juneau,AK,USA,620000
3,1300.000000,4,3,"222 Huckleberry Ln, Dillingham, AK 99576",99576,Dillingham,AK,USA,295000
4,528.000000,4,4,"Nhn Chilkat Lk, Haines, AK 99827",99827,Haines,AK,USA,120000
...,...,...,...,...,...,...,...,...,...
6301,2740.691602,6,5,"3535 Lois Dr, Henrico, VA 23229",23229,Henrico,VA,USA,6233554
6302,4179.479164,9,8,"5803 Clairmount St, Henrico, VA 23229",23229,Henrico,VA,USA,11713191222222222
6303,3304.791536,7,6,"2293 Ontario Rd NW APT 3, Washington, DC 20009...",23229,Henrico,VA,USA,10702314666666666
6304,1139.516210,1,1,"6525 Shepherd St NW, Washington, DC 20011, Ric...",23226,Richmond,VA,USA,3016983977777778


In [37]:
df2.to_csv('house_price_dataset_cleaned.csv')